# Chapter 1

### Function

```
# DRY : make functions that do only one thing at a time
# None is a good choice for default argument
# See documentation
print(func_name.__doc__)
# Alternative way
import inspect
print(inspect.getdoc(func_name))
```

# Chapter 2

### Function based Context Manager

```
@contextlib.contextmanager
def database (url):
    # set up database connection
    db = postgres.connect(url)

    try:
        yield db
    except:
        print("Something is Wrong")
    finally:
        # tear down database connection
        db.disconnect()
url = 'http://datacamp.com/data'

with database(url) as my_db:
    course_list = my_db.execute( 'SELECT * FROM courses')
```

### Class based context manager

```
    
#### Class based context manager ##########
from contextlib import contextmanager

class DatabaseManager:
    def __init__(self, url):
        self.url = url
        self.db = None

    def connect(self):
        # set up database connection
        self.db = postgres.connect(self.url)
        return self.db

    def disconnect(self):
        # tear down database connection
        if self.db:
            self.db.disconnect()

    def __enter__(self):
        # Called when entering the 'with' block
        return self.connect()

    def __exit__(self, exc_type, exc_value, traceback):
        # Called when exiting the 'with' block
        self.disconnect()

        # Handle exceptions
        if exc_type is not None:
            print("Something is Wrong")

# Example usage:
url = 'http://datacamp.com/data'

with DatabaseManager(url) as my_db:
    try:
        course_list = my_db.execute('SELECT * FROM courses')
        # Perform operations with the database connection
    except Exception as e:
        # Handle specific exceptions if needed
        print(f"Exception: {e}")


```

### Nested context

- Best practice :  Connect the nested context using a function

```
def copy(src, dst):
    """Copy the contents of one file to another.
    Args:
    src (str): File name of the file to be copied.
    dst (str): Where to write the new file.
    """
# Open both files
with open(src) as f_src:
    with open(dst, 'w') as f_dst:
    # Read and write each line, one at a time
        for line in f_src:
            f_dst.write(line)
```